In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Goal: predict whether a user will purchase an item using an e-commerce dataset

Loading data: online retail dataset from Kaggle

In [58]:
retail_data = pd.read_csv('data.csv', encoding='ISO-8859-1')
retail_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [59]:
retail_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


Exploring and cleaning the data:

InvoiceNo are either numeric (completed transaction), or start with C or A (canceled or adjusted transactions)

We will remove canceled and adjusted transactions, keeping only completed transactions

In [75]:
# Number of unique transactions
print(f'Number of unique transactions: {retail_data.InvoiceNo.unique().shape[0]:5d}')

# Transactions with missing InvoiceNo
missing_invoices = retail_data[retail_data['InvoiceNo'].isna()]
print(f"Number of transactions with missing InvoiceNo: {missing_invoices.shape[0]:1d}")

# Canceled or adjusted transactions (non-numeric InvoiceNo ; starting with C or A)
canceled_invoices = retail_data[~retail_data['InvoiceNo'].str.isnumeric()]
print(f"Number of unique canceled or adjusted transactions: {canceled_invoices['InvoiceNo'].unique().shape[0]:5d}")

# Remove canceled or adjusted transactions
cleaned_retail_data = retail_data[retail_data['InvoiceNo'].str.isnumeric()].copy()
cleaned_retail_data['InvoiceNo'] = cleaned_retail_data['InvoiceNo'].astype(int)

Number of unique transactions: 25900
Number of transactions with missing InvoiceNo: 0
Number of unique canceled or adjusted transactions:  3839


Also drop rows with missing CustomerID

In [80]:
# Find rows with missing CustomerID
missing_customerid = cleaned_retail_data[cleaned_retail_data['CustomerID'].isna()]
print(f"Number of transactions with missing CustomerID: {missing_customerid.shape[0]:1d}")

# Drop rows with missing CustomerID
cleaned_retail_data = cleaned_retail_data.dropna(subset=['CustomerID'])
cleaned_retail_data['CustomerID'] = cleaned_retail_data['CustomerID'].astype(int)

Number of transactions with missing CustomerID: 134694


More data cleaning: convert InvoiceDate to datetime format and compute total price

In [85]:
# Convert InvoiceDate to datetime
cleaned_retail_data['InvoiceDate'] = pd.to_datetime(cleaned_retail_data['InvoiceDate'], format='%m/%d/%Y %H:%M')

# Compute TotalPrice
cleaned_retail_data['TotalPrice'] = cleaned_retail_data['Quantity'] * cleaned_retail_data['UnitPrice']

In [86]:
cleaned_retail_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


In [87]:
cleaned_retail_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 397924 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    397924 non-null  int64         
 1   StockCode    397924 non-null  object        
 2   Description  397924 non-null  object        
 3   Quantity     397924 non-null  int64         
 4   InvoiceDate  397924 non-null  datetime64[ns]
 5   UnitPrice    397924 non-null  float64       
 6   CustomerID   397924 non-null  int64         
 7   Country      397924 non-null  object        
 8   TotalPrice   397924 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(3)
memory usage: 30.4+ MB
